In [56]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv

# Step 1: Create Synthetic Graph Dataset
num_nodes = 100
num_features = 1

# Generate random node features
x = torch.randn(num_nodes, num_features)

# Generate random adjacency matrix (assuming undirected graph)
edge_index = torch.randint(0, num_nodes, (2, num_nodes * 2))
edge_index = torch.unique(edge_index, dim=1)

# Create synthetic graph data
graph_data = Data(x=x, edge_index=edge_index)

train_loader = DataLoader([graph_data], batch_size = 12)

print("Data:", graph_data)
for i in train_loader:
    print("Loader:", i)

# Step 2: Define Graph Autoencoder Model
class GraphAutoencoder(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(GraphAutoencoder, self).__init__()
        self.encoder = GCNConv(input_dim, hidden_dim)
        self.decoder = GCNConv(hidden_dim, input_dim)

    def forward(self, x, edge_index):
        x = self.encoder(x, edge_index)
        x = torch.relu(x)
        x = self.decoder(x, edge_index)
        return x


def graph_autoencoder_loss(output, target_features, target_adjacency):
    features_loss = nn.MSELoss()(output, target_features)

    # Coalesce target_adjacency tensor
    target_adjacency = target_adjacency.coalesce()


    # Convert target_adjacency to dense tensor
    target_adjacency_dense = torch.sparse_coo_tensor(target_adjacency.indices(), torch.ones_like(target_adjacency.values()), size=target_adjacency.size()).to_dense()

    print(target_adjacency_dense, target_adjacency_dense.shape)

    adjacency_pred = torch.matmul(output, output.t())

    
    # Calculate BCE loss
    adjacency_loss = nn.BCEWithLogitsLoss()(adjacency_pred.flatten(), target_adjacency_dense.flatten().float())
    
    return features_loss + adjacency_loss



# Step 4: Training Loop
num_epochs = 10
hidden_dim = 2
model = GraphAutoencoder(num_features, hidden_dim)
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Assuming train_loader is a PyTorch DataLoader containing the synthetic graph data
num_batches = len(train_loader)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0  # Reset total loss for the epoch
    for batch in train_loader:
        print("Batch", batch)
        optimizer.zero_grad()
        output = model(batch.x, batch.edge_index)
        loss = graph_autoencoder_loss(output, batch.x, torch.sparse_coo_tensor(batch.edge_index, torch.ones_like(batch.edge_index[0]), size=(num_nodes, num_nodes)))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()  # Accumulate the loss for the epoch
    avg_loss = total_loss / num_batches
    print(f'Epoch [{epoch+1}/{num_epochs}], Average Loss: {avg_loss:.4f}')


# Step 5: Evaluation
# You can use the trained model to reconstruct both the features and the adjacency matrix


Data: Data(x=[100, 1], edge_index=[2, 200])
Loader: DataBatch(x=[100, 1], edge_index=[2, 200], batch=[100], ptr=[2])
Batch DataBatch(x=[100, 1], edge_index=[2, 200], batch=[100], ptr=[2])
tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]) torch.Size([100, 100])
Epoch [1/10], Average Loss: 2.9833
Batch DataBatch(x=[100, 1], edge_index=[2, 200], batch=[100], ptr=[2])
tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]) torch.Size([100, 100])
Epoch [2/10], Average Loss: 2.9146
Batch DataBatch(x=[100, 1], edge_index=[2, 200], batch=[100], ptr=[2])
tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
   

In [44]:
batch.edge_index

tensor([[ 0,  1,  1,  2,  2,  2,  3,  4,  5,  6,  6,  7,  7,  7,  7,  8,  8,  8,
          8,  9,  9,  9, 10, 10, 11, 12, 12, 12, 14, 14, 15, 15, 16, 16, 17, 18,
         18, 19, 19, 19, 19, 19, 19, 19, 20, 20, 20, 21, 21, 21, 23, 23, 23, 24,
         24, 27, 27, 28, 28, 28, 28, 28, 29, 30, 31, 32, 32, 32, 32, 33, 34, 35,
         35, 35, 36, 36, 36, 39, 40, 40, 42, 42, 43, 43, 44, 44, 45, 45, 47, 47,
         48, 48, 49, 49, 49, 49, 50, 52, 52, 53, 53, 54, 55, 55, 56, 56, 57, 57,
         59, 59, 60, 60, 60, 61, 62, 62, 62, 63, 63, 63, 63, 63, 63, 64, 65, 65,
         66, 66, 67, 67, 68, 68, 69, 69, 70, 70, 72, 72, 74, 74, 74, 75, 75, 75,
         79, 79, 80, 81, 81, 83, 83, 83, 83, 84, 84, 85, 85, 86, 86, 86, 86, 87,
         87, 87, 87, 87, 87, 88, 89, 89, 89, 89, 90, 90, 90, 91, 92, 92, 93, 93,
         94, 95, 95, 95, 95, 96, 96, 96, 96, 96, 96, 97, 97, 97, 97, 98, 99],
        [77, 68, 69, 23, 35, 53, 27, 41, 59,  4, 68, 16, 27, 82, 86,  5, 17, 43,
         55, 56, 59, 61,  9, 77

In [43]:
torch.sparse_coo_tensor(batch.edge_index, torch.ones_like(batch.edge_index[0]), size=(num_nodes, num_nodes))

tensor(indices=tensor([[ 0,  1,  1,  2,  2,  2,  3,  4,  5,  6,  6,  7,  7,  7,
                         7,  8,  8,  8,  8,  9,  9,  9, 10, 10, 11, 12, 12, 12,
                        14, 14, 15, 15, 16, 16, 17, 18, 18, 19, 19, 19, 19, 19,
                        19, 19, 20, 20, 20, 21, 21, 21, 23, 23, 23, 24, 24, 27,
                        27, 28, 28, 28, 28, 28, 29, 30, 31, 32, 32, 32, 32, 33,
                        34, 35, 35, 35, 36, 36, 36, 39, 40, 40, 42, 42, 43, 43,
                        44, 44, 45, 45, 47, 47, 48, 48, 49, 49, 49, 49, 50, 52,
                        52, 53, 53, 54, 55, 55, 56, 56, 57, 57, 59, 59, 60, 60,
                        60, 61, 62, 62, 62, 63, 63, 63, 63, 63, 63, 64, 65, 65,
                        66, 66, 67, 67, 68, 68, 69, 69, 70, 70, 72, 72, 74, 74,
                        74, 75, 75, 75, 79, 79, 80, 81, 81, 83, 83, 83, 83, 84,
                        84, 85, 85, 86, 86, 86, 86, 87, 87, 87, 87, 87, 87, 88,
                        89, 89, 89, 89, 

In [63]:
coo_indices = torch.randint(0, 3, (2, 5))  # Assuming 1000 nodes
print(coo_indices)
# Number of nodes in the graph
num_nodes = 3

# Create an empty adjacency matrix
adj_matrix = torch.zeros(num_nodes, num_nodes)

# Fill adjacency matrix using COO tensor
adj_matrix[coo_indices[0], coo_indices[1]] = 1
adj_matrix[coo_indices[1], coo_indices[0]] = 1
adj_matrix

tensor([[0, 2, 1, 0, 1],
        [1, 1, 0, 0, 1]])


tensor([[1., 1., 0.],
        [1., 1., 1.],
        [0., 1., 0.]])

In [78]:
x = torch.tensor([1,2,3])
torch.ger(x, x)

tensor([[1, 2, 3],
        [2, 4, 6],
        [3, 6, 9]])